In [1]:
import torch
import torch.nn as nn

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
torch.cuda.is_available()

False

In [4]:
input_size = 6
hidden_size = 500 
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        sample = {
            'date': self.data.iloc[idx, 0], 
            'close': self.data.iloc[idx, 2],
            'open': self.data.iloc[idx, 5], 
            'high': self.data.iloc[idx, 3], 
            'low': self.data.iloc[idx, 4],
            'volume': self.data.iloc[idx, 6]
        }

        if self.transform:
            sample = self.transform(sample)

        return sample


btc_csv_file = './data/BTC-USD-1d.csv'
btc_custom_dataset = CustomDataset(btc_csv_file)

eth_csv_file = './data/ETH-USD-1d.csv'
eth_custom_dataset = CustomDataset(eth_csv_file)

xrp_csv_file = './data/XRP-USD-1d.csv'
xrp_custom_dataset = CustomDataset(xrp_csv_file)

ltc_csv_file = './data/LTC-USD-1d.csv'
ltc_custom_dataset = CustomDataset(ltc_csv_file)

doge_csv_file = './data/DOGE-USD-1d.csv'
doge_custom_dataset = CustomDataset(doge_csv_file)

In [6]:
from sklearn.model_selection import train_test_split

btc_train_indices, btc_test_indices = train_test_split(range(len(btc_custom_dataset)), test_size=0.2, random_state=42)
eth_train_indices, eth_test_indices = train_test_split(range(len(eth_custom_dataset)), test_size=0.2, random_state=42)
xrp_train_indices, xrp_test_indices = train_test_split(range(len(xrp_custom_dataset)), test_size=0.2, random_state=42)
ltc_train_indices, ltc_test_indices = train_test_split(range(len(ltc_custom_dataset)), test_size=0.2, random_state=42)
doge_train_indices, doge_test_indices = train_test_split(range(len(doge_custom_dataset)), test_size=0.2, random_state=42)

In [7]:
btc_train_loader = torch.utils.data.DataLoader(dataset=btc_train_indices, batch_size=batch_size)
eth_train_loader = torch.utils.data.DataLoader(dataset=eth_train_indices, batch_size=batch_size)
xrp_train_loader = torch.utils.data.DataLoader(dataset=xrp_train_indices, batch_size=batch_size)
ltc_train_loader = torch.utils.data.DataLoader(dataset=ltc_train_indices, batch_size=batch_size)
doge_train_loader = torch.utils.data.DataLoader(dataset=doge_train_indices, batch_size=batch_size)


btc_test_loader = torch.utils.data.DataLoader(dataset=btc_test_indices, batch_size=batch_size)
eth_test_loader = torch.utils.data.DataLoader(dataset=eth_test_indices, batch_size=batch_size)
xrp_test_loader = torch.utils.data.DataLoader(dataset=xrp_test_indices, batch_size=batch_size)
ltc_test_loader = torch.utils.data.DataLoader(dataset=ltc_test_indices, batch_size=batch_size)
doge_test_loader = torch.utils.data.DataLoader(dataset=doge_test_indices, batch_size=batch_size)

train_loader = [btc_train_loader, eth_train_loader, xrp_train_loader, 
                ltc_train_loader, doge_train_loader]
test_loader = [btc_test_loader, eth_test_loader, xrp_test_loader, 
               ltc_test_loader, doge_test_loader]

In [9]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

In [11]:
total_step = len(btc_train_loader)
for epoch in range(num_epochs):
    for i, data in enumerate(btc_train_loader):
        # Extract data
        # Note: Adjust this part depending on how you've structured your data loader
        # You might need to handle different datasets separately if they have different structures
        inputs, labels = data['input'], data['target']
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

print('Finished Training')

IndexError: too many indices for tensor of dimension 1

In [ ]:
# Evaluation
model.eval()  # Set the model to evaluation mode

total_correct = 0
total_samples = 0
total_loss = 0

with torch.no_grad():
    for data in btc_test_loader:
        inputs, labels = data['input'], data['target']
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        total_loss += loss.item()
        
        _, predicted = torch.max(outputs, 1)
        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

    # Calculate average loss and accuracy
    avg_loss = total_loss / len(btc_test_loader)
    accuracy = 100 * total_correct / total_samples
    print(f'Test Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%')

print('Finished Testing')